Google Colab: https://colab.research.google.com/drive/1OpWlDGd6HnD4bmzEtywARctu2riKfFJk?usp=sharing

HuggingFace: https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3

## Libraries Explained

- **dotenv**: Loads environment variables from a `.env` file into the application's environment, helping manage configuration separately from code.

- **huggingface_hub**: 
  - **HfApi**: Provides programmatic access to the Hugging Face model hub for uploading, downloading, and managing models.
  - **hf_hub_download**: Simplifies downloading model files from the Hugging Face hub to your local environment.

- **transformers**: Offers pre-trained models for natural language processing tasks. The `pipeline` function specifically provides an easy-to-use interface for common NLP tasks like text generation, sentiment analysis, and question answering.


In [1]:
import os, json, datetime
from datetime import datetime
from dotenv import load_dotenv

from huggingface_hub import HfApi
from huggingface_hub import hf_hub_download



from transformers import pipeline


# Loading Environment Variables for Hugging Face


This code snippet performs two essential operations:

1. `load_dotenv()` - Loads environment variables from a `.env` file into the application's environment. This is a common pattern for securely storing configuration and sensitive information outside of the source code.

2. `hf_key = os.getenv("HF_TOKEN")` - Retrieves the Hugging Face API token from the environment variables and assigns it to the variable `hf_key`. This token is required for authenticated access to the Hugging Face Hub services, including downloading private models or models with gated access.


In [2]:
load_dotenv()
hf_key=os.getenv("HF_TOKEN")


# Hugging Face Model Reference

[mistralai/Mistral-7B-Instruct-v0.3](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3)

# Mistral 7B Instruct Model

## Model Overview

This reference points to Mistral AI's 7B-parameter instruction-tuned language model, version 0.3, hosted on the Hugging Face model hub.

## Key Specifications

- **Architecture**: Transformer-based language model
- **Size**: 7 billion parameters (compact but powerful)
- **Type**: Instruction-tuned generative model
- **Version**: 0.3 (latest stable release)
- **Developer**: Mistral AI
- **License**: Apache 2.0

## Model Capabilities

- Follows natural language instructions
- Generates coherent and contextually relevant text
- Handles various tasks through prompting:
  - Question answering
  - Summarization
  - Creative writing
  - Explanations
  - Reasoning
  - Code generation

## Technical Features

- Efficient architecture with strong performance for its size
- Sliding Window Attention mechanism for handling longer contexts
- Grouped-query attention for improved inference speed
- Suitable for deployment on consumer-grade hardware

This model represents a good balance between capability and resource requirements, making it accessible for various applications while delivering performance competitive with much larger models.

In [3]:
hf_reference='mistralai/Mistral-7B-Instruct-v0.3'


# Making API Calls to Mistral 7B Instruct


## Function Explanation

This code demonstrates how to interact with the Mistral 7B Instruct model through Hugging Face's Inference API:

### API Configuration
- **Endpoint**: Uses Hugging Face's router API with Together AI as the provider
- **Authentication**: Bearer token authorization with your Hugging Face API key
- **Model**: Specifies Mistral 7B Instruct v0.3

### Query Function
- **Purpose**: Encapsulates the API call logic for reusability
- **Input**: Takes a payload dictionary with messages and parameters
- **Process**: Sends a POST request to the API endpoint
- **Output**: Returns the JSON response from the API

### Request Structure
- **Messages Array**: Contains conversation history in a chat format
  - Each message has a "role" (user/assistant) and "content"
  - This example has a single user message asking about France's capital
- **Max Tokens**: Limits the response length to 512 tokens
- **Model**: Explicitly specifies which model to use

### Response Handling
- Extracts the text response from the nested JSON structure
- `response["choices"][0]["message"]["content"]` accesses the actual text reply

This approach follows the standard chat completions API format used by many modern LLMs, making it adaptable to other models with minimal changes.

In [4]:
import requests

API_URL = "https://router.huggingface.co/together/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {hf_key}",
}

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

response = query({
    "messages": [
        {
            "role": "user",
            "content": "write a code to explain recursion in python?"
        }
    ],
    "max_tokens": 1024,
    "model": "mistralai/Mistral-7B-Instruct-v0.3"
})

print(response["choices"][0]["message"]["content"])

 Here's a simple example of recursion in Python using the Fibonacci sequence. This code defines a function `fibonacci(n)` that calculates the nth number in the Fibonacci sequence.

```python
def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)

# Test the function
print(fibonacci(10))  # Output: 55
```

In this example, the base case is when `n` is less than or equal to 1, where the function returns `n`. For other values of `n`, the function calls itself recursively with `n-1` and `n-2` as arguments until it reaches the base case.

When you run this code, it will calculate the 10th Fibonacci number (55) by repeatedly calling the `fibonacci` function. Each call to the function will eventually reach the base case and return a value, which will be added to the return value of the previous call until the final result is calculated. This demonstrates the concept of recursion in Python.



# Serialize and Save Model Information from Hugging Face Hub


This code demonstrates how to retrieve, serialize, and save detailed model information from the Hugging Face Hub:

1. **Serialization Function**: The `serialize_object()` function handles complex objects recursively:
   - Converts datetime objects to ISO format strings
   - Transforms objects with `__dict__` attributes into dictionaries
   - Processes nested lists and dictionaries
   - Preserves primitive data types

2. **API Interaction**: Creates an instance of the Hugging Face API client

3. **Model Information**: Fetches comprehensive metadata about the specified model using `api.model_info()`

4. **File Operations**: 
   - Extracts the model name from the reference path
   - Creates a JSON file named after the model
   - Serializes the model information and writes it to the file

This allows for local storage of model metadata for later reference or analysis, particularly useful for model governance, versioning, and documentation purposes.


In [5]:
def serialize_object(obj):
    """
    Helper function to serialize custom objects like EvalResult.
    Converts objects with __dict__ attribute to dictionaries and handles datetime objects.
    """
    if isinstance(obj, datetime):
        return obj.isoformat()  # Convert datetime to ISO 8601 string
    elif hasattr(obj, "__dict__"):
        return {key: serialize_object(value) for key, value in obj.__dict__.items()}
    elif isinstance(obj, list):
        return [serialize_object(item) for item in obj]
    elif isinstance(obj, dict):
        return {key: serialize_object(value) for key, value in obj.items()}
    else:
        return obj  # Return the value as-is for primitive types

api = HfApi()
with open(f"models/{hf_reference.split('/')[1]}.json", "w") as json_file:
    json_file.write(json.dumps(serialize_object(api.model_info(hf_reference))))
